In [1]:
import logging

logging.basicConfig(level=logging.INFO)


In [2]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss, MSELoss

# from pennylane import NesterovMomentumOptimizer

from qcc.ml.data import Data, BinaryData
from qcc.ml.optimize import Optimizer
from qcc.ml.data import ImageTransform, ImageTransform1D, ClassicalImageTransform
from qcc.experiment import Experiment
from qcc.ml.model import Model

from pathlib import Path
from qcc.file import new_dir

# from qcc.quantum.pennylane.ansatz import MQCCOptimized as Module
# from qcc.quantum.pennylane.ansatz import QCNN as Module
# from qcc.ml.cnn import ConvolutionalNeuralNetwork as Module
# from qcc.ml.mlp import MultiLayerPerceptron as Module
# from qcc.quantum.quanvolution import QuanvolutionalNeuralNetwork as Module
from qcc.ml.hybrid_cnn import MQCCNonHybrid as Module

# from qcc.quantum.pennylane.ansatz.basic import BasicFiltering6
from qcc.quantum.pennylane.pyramid import Pyramid
from qcc.quantum.pennylane.c2q import ConvolutionAngleFilter, ConvolutionComplexAngleFilter, ConvolutionFilter
from qcc.quantum.pennylane.local import define_filter


INFO:qcc.ml.ml:Using CUDA on device cuda


In [3]:
# Meta parameters
name = "sdfljhvosdv"
filename = Path(f"results/{name}")
num_trials = 1
silent = False
is_quantum = False

# Ansatz parameters
dims = (28, 28, 3, 1)
num_layers = 5
module_options = {
    # "U_filter": ConvolutionAngleFilter,
    # "pre_op": True,
    "num_features": 4,
    # "U_fully_connected": ConvolutionAngleFilter,
    # "pooling": True,
    # # "filter_shape": (2, 2, 2),
    # "q2c_method": "parity"
    # "bias": True
}


In [4]:
filename = new_dir(filename, overwrite=True)
filename = filename / name

if is_quantum:
    module = Module.from_dims
else:
    module = Module

# Create module
module = module(
    dims,
    num_layers=num_layers,
    **module_options,
)
data = BinaryData(
    CIFAR10,
    ImageTransform(dims) if is_quantum else ClassicalImageTransform(dims),
    batch_size=(8,8),
)
optimizer = Optimizer(Adam)
loss = CrossEntropyLoss()
model = Model.with_logging(module, data, optimizer, loss, epoch=8)

# Log important values
model.logger.info(f"Circuit ID: {name}")
model.logger.info(f"{module=}")
model.logger.info(f"{data=}")
model.logger.info(f"{optimizer=}")
model.logger.info(f"{loss=}")

model.logger.info(f"{num_trials=}")
model.logger.info(f"{dims=}")
model.logger.info(f"{num_layers=}")
# model.logger.info(f"{1=}")
model.logger.info(f"{module_options=}")

# Save circuit drawing
if is_quantum:
    filename = filename.with_stem(f"{name}_circuit")
    module.draw(filename=filename, decompose=True)


INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 39
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 188
INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 13
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 56
INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 8
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 29
INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 7
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 21
INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 6
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 15
INFO:qcc.quantum.pennylane.ansatz.ansatz:Depth: 9
INFO:qcc.quantum.pennylane.ansatz.ansatz:Gate Count: 19
2023-11-06 00:12:48,708: (mqccnonhybrid) Circuit ID: sdfljhvosdv
2023-11-06 00:12:48,709: (mqccnonhybrid) module=MQCCNonHybrid(
  (0): MQCCLayer(
    (mqcc): MQCC()
  )
  (1): MQCCLayer(
    (mqcc): MQCC()
  )
  (2): MQCCLayer(
    (mqcc): MQCC()
  )
  (3): MQCCLayer(
    (mqcc): MQCC()
  )
  (4): MQCCLayer(
    (mqcc): MQCC()
  )
  (5): Flatten(start_dim=1, 

In [5]:
# Run experiment
results_schema = ["accuracy", "training_time", "testing_time"]
experiment = Experiment(model, num_trials, results_schema)
# experiment.partial(silent=silent)
results = experiment(filename=filename)


Files already downloaded and verified
Files already downloaded and verified


2023-11-06 00:12:49,781: (mqccnonhybrid) Number of Parameters: 74
/home/mingyoungjeng/Documents/repos/quantum-convolutional-classifier/src/qcc/quantum/quantum.py:125: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3614.)
  data = data.reshape(new_dims[::-1]).T


KeyboardInterrupt: 

In [ ]:
# Print accuracy results
metrics = ("median", "mean", "max", "min", "std")
for name in results.columns:
    col = results[name]
    msg = (f"{metric}={getattr(col, metric)()}" for metric in metrics)
    msg = ", ".join(msg)
    msg = f"{name}: {msg}"
    model.logger.info(msg)

# Save aggregated loss history figure
display(experiment.draw(filename))
